# Train pipeline

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 셋팅

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

## 2. training pipeline 만들기
### Pipeline:
* https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-pipelines/index.html 
* https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-pipeline-parameterization/parameterized-pipeline.html 
* https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#id2

### Increase quotas for ResourceLimitExceeded Error
* https://aws.amazon.com/ko/premiumsupport/knowledge-center/sagemaker-resource-limit-exceeded-error/

### Retry Policy
* https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-retry-policy.html
* parameters <br>
    * *expire_after_mins:* <br>
         - A positive integer that represents the maximum timespan of retry. <BR>
         - If the error recurs after ExpireAfterMin minutes counting from the step gets executed, retries cease and normal error handling resumes. <BR>
         - A value of 0 specifies that errors are never retried. ExpireAfterMin has a maximum value of 14,400 minutes (10 days) <BR>
    * *interval_seconds:* <br>
        - (optional)The number of seconds before the first retry attempt (1 by default). IntervalSeconds has a maximum value of 43200 seconds (12 hours). <BR>
    * *backoff_rate:* <br>
        - (optional): The multiplier by which the retry interval increases during each attempt (2.0 by default).

In [4]:
import sagemaker
from pprint import pprint
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.functions import Join
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.tensorflow import TensorFlowProcessor
from sagemaker.workflow.properties import PropertyFile
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.pytorch.processing import PyTorchProcessor
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.workflow.retry import StepRetryPolicy, StepExceptionTypeEnum, SageMakerJobExceptionTypeEnum, SageMakerJobStepRetryPolicy

In [11]:
class train_pipeline():
    
    def __init__(self, model_name): 
        
        self.strModelName = model_name
        self.strRegionName = boto3.Session().region_name
        self.pm = parameter_store(self.strRegionName)
        strPrefix = pm.get_params(key="PREFIX")
        
        self.account_id = pm.get_params(key=strPrefix + "ACCOUNT-ID")
        self.default_bucket = self.pm.get_params(key=strPrefix + "DEFAULT-BUCKET") # sagemaker.Session().default_bucket()
        self.role = self.pm.get_params(key=strPrefix + "SAGEMAKER-ROLE-ARN")#self.args.config.get_value("ROLE", "sagemaker_execution_role")
        self.strInOutPrefix = '/opt/ml/processing' 
        self.pipeline_session = PipelineSession()
        self.cache_config = CacheConfig(
            enable_caching=False,
            expire_after="T48H"
        )
        self.pipeline_name = f"train-pipeline-{self.strModelName}"
        self.model_image_uri = sagemaker.image_uris.retrieve("blazingtext", self.strRegionName)
        
        print (f"MODEL-NAME: {self.strModelName}")
        print (f'  Account-ID: {self.account_id}, \n  Region: {self.strRegionName}, \n  Role: {self.role}, \n  Default_bucket: {self.default_bucket}')
        print (f'  pipeline_session: {self.pipeline_session}')
         
    def _step_preprocessing(self, ):
        
        sklearn_processor = SKLearnProcessor(
            framework_version="0.20.0",
            role=self.role,
            instance_type="ml.m5.xlarge",
            instance_count=1,
            base_job_name="preprocessing", # bucket에 보이는 이름 (pipeline으로 묶으면 pipeline에서 정의한 이름으로 bucket에 보임)
            sagemaker_session=self.pipeline_session ## Add
        )
        
        step_args = sklearn_processor.run(
            code='./sources/preprocessing/preprocessing.py',
            inputs=[
                ProcessingInput(
                    source=f"s3://{pm.get_params(key=strPrefix + 'DATA-BUCKET')}/{self.strModelName}/amazon_reviews_us_Electronics_v1_00.tsv.gz",
                    destination='/opt/ml/processing/input'
                ),
                ProcessingInput(
                    input_name="requirements",
                    source='./sources/preprocessing/requirements.txt',
                    destination="/opt/ml/processing/input/requirements"
                ),
            ],
            outputs=[
                ProcessingOutput(
                    output_name="train_data",
                    source='/opt/ml/processing/output/train',
                    destination=Join(
                        on="/",
                        values=[
                            "s3://{}".format(self.default_bucket),
                            self.pipeline_name,
                            ExecutionVariables.PROCESSING_JOB_NAME,
                            "train_data",
                        ],
                    ),
                ),
                ProcessingOutput(
                    output_name="validation_data",
                    source='/opt/ml/processing/output/validation',\
                    destination=Join(
                        on="/",
                        values=[
                            "s3://{}".format(self.default_bucket),
                            self.pipeline_name,
                            ExecutionVariables.PROCESSING_JOB_NAME,
                            "validation_data",
                        ],
                    ),
                ),
                ProcessingOutput(
                    output_name="test_data",
                    source='/opt/ml/processing/output/test',
                    destination=Join(
                        on="/",
                        values=[
                            "s3://{}".format(self.default_bucket),
                            self.pipeline_name,
                            ExecutionVariables.PROCESSING_JOB_NAME,
                            "test_data",
                        ],
                    ),
                )
            ],
            arguments=["--input_name", "amazon_reviews_us_Electronics_v1_00.tsv.gz", "--region", strRegionName],
            job_name=f"preprocessing-{self.strModelName}",
        )
        
        self.preprocessing_process = ProcessingStep(
            name="PreprocessingProcess", ## Processing job이름
            step_args=step_args,
            cache_config=self.cache_config,
            retry_policies=[                
                # retry when resource limit quota gets exceeded
                SageMakerJobStepRetryPolicy(
                    exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
                    expire_after_mins=180,
                    interval_seconds=600,
                    backoff_rate=1.0
                ),
            ]
        )
        
        print ("  \n== Preprocessing Step ==")
        print ("   \nArgs: ", self.preprocessing_process.arguments.items())
        
    def _step_training(self, ):
        
        max_jobs = 6
        max_parallel_jobs = 3
        objective_name = "validation:accuracy"
        
        self.estimator=Estimator(
            image_uri=self.model_image_uri,
            role=self.role, 
            instance_count=1,
            instance_type="ml.m5.xlarge",
            volume_size=30,
            max_run=360000,
            input_mode= 'File',
            output_path=Join(
                on="/",
                values=[
                    "s3://{}".format(self.default_bucket),
                    self.pipeline_name,
                ],
            ),
            sagemaker_session=self.pipeline_session ## add
        )

        self.estimator.set_hyperparameters(
            mode="supervised",
            epochs=10,
            min_epochs=5, # Min epochs before early stopping is introduced
            early_stopping=True,
            patience=2,
            learning_rate=0.01,
            min_count=2, # words that appear less than min_count are discarded 
            word_ngrams=2, # the number of word n-gram features to use.
            vector_dim=16, # dimensions of embedding layer
        ) 
        
        step_training_args = self.estimator.fit(
            inputs={
                "train": TrainingInput(
                    s3_data=self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["train_data"].S3Output.S3Uri,
                    content_type="text/csv"
                ),
                "validation": TrainingInput(
                    s3_data=self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["validation_data"].S3Output.S3Uri,
                    content_type="text/csv"
                )
            },
            logs="All",
        )
          
        self.training_process = TrainingStep(
            name=f"TrainingProcess",
            step_args=step_training_args,
            cache_config=self.cache_config,
            depends_on=[self.preprocessing_process],
            retry_policies=[                
                # retry when resource limit quota gets exceeded
                SageMakerJobStepRetryPolicy(
                    exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
                    expire_after_mins=180,
                    interval_seconds=600,
                    backoff_rate=1.0
                ),
            ]
        )
        
        print ("  \n== Training Step ==")
        print ("   \nArgs: ", self.training_process.arguments.items())
        
    def _step_evaluation(self, ):
        
        eval_processor = TensorFlowProcessor(
            framework_version='2.3',
            role=self.role,
            instance_type="ml.g4dn.xlarge", 
            instance_count=1,
            base_job_name='evaluation',
            py_version='py37',
            sagemaker_session=self.pipeline_session
        )
        
        step_args = eval_processor.run(
            job_name="evaluation", # Evaluation job name. If not specified, the processor generates a default job name, based on the base job name and current timestamp.
                                   # 이걸 넣어야 캐시가 작동함, 안그러면 프로세서의 base_job_name 이름뒤에 날짜 시간이 붙어서 캐시 동작 안함
            code='./evaluation/evaluation.py', #소스 디렉토리 안에서 파일 path
            source_dir="./sources/", #현재 파일에서 소스 디렉토리 상대경로 # add script.py and requirements.txt here
            
            inputs=[
                ProcessingInput(
                    source=self.training_process.properties.ModelArtifacts.S3ModelArtifacts,
                    destination="/opt/ml/processing/model"
                ), ## 모델을 
                ProcessingInput(
                    source=self.preprocessing_process.properties.ProcessingOutputConfig.Outputs["test_data"].S3Output.S3Uri,
                    destination="/opt/ml/processing/test"
                )
            ],
            outputs=[
                ProcessingOutput(
                    output_name="evaluation-metrics",
                    source="/opt/ml/processing/evaluation",
                    destination=Join(
                        on="/",
                        values=[
                            "s3://{}".format(self.default_bucket),
                            self.pipeline_name,
                            ExecutionVariables.PROCESSING_JOB_NAME,
                            "evaluation-metrics",
                        ],
                    ),
                )
            ],
            arguments=["--s3_model_path", self.training_process.properties.ModelArtifacts.S3ModelArtifacts, "--region", self.strRegionName, "--model_name", self.strModelName]
        )

        self.evaluation_process = ProcessingStep(
            name="EvaluationProcess", ## Processing job이름
            step_args=step_args,
            cache_config=self.cache_config,
            depends_on=[self.preprocessing_process, self.training_process],
            retry_policies=[                
                # retry when resource limit quota gets exceeded
                SageMakerJobStepRetryPolicy(
                    exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
                    expire_after_mins=180,
                    interval_seconds=600,
                    backoff_rate=1.0
                ),
            ]
        )
        
        print ("  \n== Evaluation Step ==")
        print ("   \nArgs: ", self.evaluation_process.arguments.items())
        
    def _step_model_registration(self, ):
      
        self.model_package_group_name = self.strModelName
                                                                              
        model_metrics = ModelMetrics(
            model_statistics=MetricsSource(
                s3_uri=Join(
                    on="/",
                    values=[
                        self.evaluation_process.properties.ProcessingOutputConfig.Outputs["evaluation-metrics"].S3Output.S3Uri, #print (self.evaluation_process.arguments.items())로 확인가능
                        f"evaluation-{self.strModelName}.json"
                    ],
                ),
                content_type="application/json")
        )
                                       
        self.register_process = RegisterModel(
            name="ModelRegisterProcess", ## Processing job이름
            estimator=self.estimator,
            image_uri=self.evaluation_process.properties.AppSpecification.ImageUri, #self.model_image_uri, 
            model_data=self.training_process.properties.ModelArtifacts.S3ModelArtifacts,
            content_types=["text/csv"],
            response_types=["text/csv"],
            inference_instances=["ml.g4dn.xlarge"],
            transform_instances=["ml.g4dn.xlarge"],
            model_package_group_name=self.model_package_group_name,
            approval_status="PendingManualApproval", ## “Approved”, “Rejected”, or “PendingManualApproval” (default: “PendingManualApproval”).
            model_metrics=model_metrics,
            depends_on=[self.evaluation_process]
        )
        
        print ("  \n== Registration Step ==")
        
    def _get_pipeline(self, ):
        
        pipeline = Pipeline(
            name=self.pipeline_name,
            steps=[self.preprocessing_process, self.training_process, self.evaluation_process, self.register_process],
            sagemaker_session=self.pipeline_session
        )

        return pipeline
                            
    def execution(self, ):
        
        self._step_preprocessing()
        self._step_training()
        self._step_evaluation()
        self._step_model_registration()
        
        pipeline = self._get_pipeline()
        pipeline.upsert(role_arn=self.role) ## Submit the pipeline definition to the SageMaker Pipelines service 
        execution = pipeline.start()
        print ("  \n== PipeLine Execution info.==")
        print (execution.describe())
        
        print ("========================== \n")

## 3. training pipeline 수행하기

In [12]:
nModels = 30

In [13]:
for nModelIdx in range(nModels):
    strModelName = f"model-{nModelIdx+1}"
    pipe = train_pipeline(model_name=strModelName)
    pipe.execution()

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


MODEL-NAME: model-1
  Account-ID: 419974056037, 
  Region: ap-northeast-2, 
  Role: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221004T162466, 
  Default_bucket: sagemaker-ap-northeast-2-419974056037
  pipeline_session: <sagemaker.workflow.pipeline_context.PipelineSession object at 0x7f518571e340>
  
== Preprocessing Step ==


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.processing:Uploaded ./sources/ to s3://sagemaker-ap-northeast-2-419974056037/evaluation/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-northeast-2-419974056037/evaluation/source/runproc.sh



Job Name:  preprocessing-model-1
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sm-pipeline-multi-models-datasets/model-1/amazon_reviews_us_Electronics_v1_00.tsv.gz', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'requirements', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/preprocessing-model-1/input/requirements/requirements.txt', 'LocalPath': '/opt/ml/processing/input/requirements', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/preprocessing-model-1/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionT

INFO:sagemaker.processing:Uploaded ./sources/ to s3://sagemaker-ap-northeast-2-419974056037/train-pipeline-model-1/code/eabace17c9345271cf901107138b46a8/sourcedir.tar.gz


...


Job Name:  preprocessing-model-30
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sm-pipeline-multi-models-datasets/model-30/amazon_reviews_us_Electronics_v1_00.tsv.gz', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'requirements', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/preprocessing-model-30/input/requirements/requirements.txt', 'LocalPath': '/opt/ml/processing/input/requirements', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/preprocessing-model-30/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistribut

INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-northeast-2-419974056037/evaluation/source/runproc.sh



Job Name:  evaluation
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': <sagemaker.workflow.properties.Properties object at 0x7f5184c925b0>, 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': <sagemaker.workflow.properties.Properties object at 0x7f5185095100>, 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/evaluation/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False

INFO:sagemaker.processing:Uploaded ./sources/ to s3://sagemaker-ap-northeast-2-419974056037/train-pipeline-model-30/code/eabace17c9345271cf901107138b46a8/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-ap-northeast-2-419974056037/train-pipeline-model-30/code/1071dadab36d48ed1fba97e2ff137bdf/runproc.sh



Job Name:  preprocessing-model-30
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sm-pipeline-multi-models-datasets/model-30/amazon_reviews_us_Electronics_v1_00.tsv.gz', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'requirements', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/preprocessing-model-30/input/requirements/requirements.txt', 'LocalPath': '/opt/ml/processing/input/requirements', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-419974056037/train-pipeline-model-30/code/2b3c51df48188408561910efa21ddc30/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMod

## Appendix - model card

sagemaker model card
* https://aws.amazon.com/ko/blogs/machine-learning/improve-governance-of-your-machine-learning-models-with-amazon-sagemaker/
* deep dive: https://aws.amazon.com/ko/blogs/aws/new-ml-governance-tools-for-amazon-sagemaker-simplify-access-control-and-enhance-transparency-over-your-ml-projects/
* https://docs.aws.amazon.com/sagemaker/latest/dg/model-cards-sdk.html
* https://sagemaker.readthedocs.io/en/stable/api/governance/model_card.html
* example: https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_governance/model_card.html

pip install sagemaker --upgrade 해주기